#Change Detection with Instance Segmentation

This notebook demonstrates the change detection functionality in GeoAI, which provides instance segmentation and confidence scoring for individual change objects.

The change detection functionality builds upon the torchange package developed by Dr. Zhuo Zheng. We have made it much easier to analyze remote sensing imagery and visualize the results.

#Overview
The change detection system provides:

* Instance Segmentation: Each change object gets a unique ID
* Confidence Scores: Individual confidence values for each detected instance
* Proper GeoTIFF Output: Maintains spatial reference information

#Key Features
* Instance-level change detection with unique IDs
* Confidence scoring for quality assessment
* Support for large GeoTIFF files
* Comprehensive analysis capabilities

In [1]:
%pip install geoai-py
import geoai
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.1/186.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.0/93.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 574.4/574.4 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/2

In [2]:
# Check if CUDA is available
device = geoai.get_device()
print(f"Using device: {device}")

# Set up paths
out_folder = "change_detection_results"
Path(out_folder).mkdir(exist_ok=True)

print(f"Working directory: {out_folder}")

Using device: cpu
Working directory: change_detection_results


In [3]:
# Download NAIP imagery
naip_2019_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/las_vegas_naip_2019_a.tif"
naip_2022_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/las_vegas_naip_2022_a.tif"

naip_2019_path = geoai.download_file(naip_2019_url)
naip_2022_path = geoai.download_file(naip_2022_url)

print(f"Downloaded 2019 NAIP: {naip_2019_path}")
print(f"Downloaded 2022 NAIP: {naip_2022_path}")

Downloaded 2019 NAIP: las_vegas_naip_2019_a.tif
Downloaded 2022 NAIP: las_vegas_naip_2022_a.tif


In [4]:
# Check raster information
geoai.get_raster_info(naip_2019_path)

{'driver': 'GTiff',
 'width': 2721,
 'height': 1420,
 'count': 4,
 'dtype': 'uint8',
 'crs': 'EPSG:26911',
 'transform': Affine(0.5999999999999914, 0.0, 649647.6,
        0.0, -0.6, 4020075.0),
 'bounds': BoundingBox(left=649647.6, bottom=4019223.0, right=651280.2, top=4020075.0),
 'resolution': (0.5999999999999914, 0.6),
 'nodata': None,
 'band_stats': [{'band': 1,
   'min': 9.0,
   'max': 227.0,
   'mean': 157.28663447054987,
   'std': 42.10052755042448},
  {'band': 2,
   'min': 14.0,
   'max': 225.0,
   'mean': 150.56818200640816,
   'std': 40.031556546614866},
  {'band': 3,
   'min': 24.0,
   'max': 217.0,
   'mean': 137.28806388496358,
   'std': 35.24497559603796},
  {'band': 4,
   'min': 6.0,
   'max': 234.0,
   'mean': 146.37050147263588,
   'std': 48.39003888698475}]}

In [5]:
# View the images
geoai.view_raster(naip_2019_path)

In [6]:
geoai.view_raster(naip_2022_path)

#Initialize Change Detection
###Create the change detection system with optimal parameters.

In [7]:
# Make sure model directory exists
Path("~/.cache/torch/hub/checkpoints/").expanduser().mkdir(parents=True, exist_ok=True)

# Initialize change detection
detector = geoai.ChangeDetection(sam_model_type="vit_h")

# Configure parameters (following the torchange example)
detector.set_hyperparameters(
    change_confidence_threshold=145,
    use_normalized_feature=True,
    bitemporal_match=True,
)

detector.set_mask_generator_params(
    points_per_side=32,
    stability_score_thresh=0.95,
)

print("Change detection system initialized!")

Model checkpoint for vit_h not found.


Extracted to: /root/.cache/torch/hub/checkpoints/sam_vit_h_4b8939
Change detection system initialized!


#Run Change Detection
###Execute change detection with instance segmentation and confidence scoring

In [ ]:
# Run change detection
results = detector.detect_changes(
    naip_2019_path,
    naip_2022_path,
    output_path=f"{out_folder}/binary_mask.tif",
    export_probability=True,
    probability_output_path=f"{out_folder}/probability_mask.tif",
    export_instance_masks=True,
    instance_masks_output_path=f"{out_folder}/instance_masks.tif",
    return_detailed_results=True,
    return_results=False,
)

print(f"Change detection completed!")
print(f"Total instances detected: {results['summary']['total_masks']}")
print(f"Image size: {results['summary']['original_shape']}")

Saved instance segmentation mask with 179 instances to change_detection_results/instance_masks.tif
Saved instance scores mask with 179 instances to change_detection_results/instance_masks_scores.tif
Change detection completed!
Total instances detected: 179
Image size: (1419, 2721)


#Analyze Results
###Display key statistics and quality metrics.

In [ ]:
# Display statistics
if "statistics" in results and results["statistics"]:
    print("Quality Statistics:")
    for metric, stats in results["statistics"].items():
        print(f"  {metric}: mean={stats['mean']:.3f}, std={stats['std']:.3f}")

# Show top instances
if "masks" in results and len(results["masks"]) > 0:
    print("\nTop 5 detected instances:")
    for i, mask in enumerate(results["masks"][:5]):
        print(
            f"  {i+1}. Instance {mask['mask_id']}: "
            f"IoU={mask['iou_pred']:.3f}, "
            f"Stability={mask['stability_score']:.3f}, "
            f"Area={mask['area']} pixels"
        )

#Visualizations
###Use the integrated visualization methods for comprehensive analysis.

In [ ]:
# probability visualization
detector.visualize_results(
    naip_2019_path,
    naip_2022_path,
    f"{out_folder}/binary_mask.tif",
    f"{out_folder}/probability_mask.tif",
)

In [ ]:
# Create split comparison visualization
detector.create_split_comparison(
    naip_2019_path,
    naip_2022_path,
    f"{out_folder}/binary_mask.tif",
    f"{out_folder}/probability_mask.tif",
    f"{out_folder}/split_comparison.png",
)

In [ ]:
# Analyze individual instances
instance_stats = detector.analyze_instances(
    f"{out_folder}/instance_masks.tif",
    f"{out_folder}/instance_masks_scores.tif",
    f"{out_folder}/instance_analysis.png",
)

#Comprehensive Analysis Report
###Generate a detailed analysis report combining all metrics

In [ ]:
# Create comprehensive analysis report
detector.create_comprehensive_report(results, f"{out_folder}/comprehensive_report.png")

# Create comprehensive analysis report
detector.create_comprehensive_report(results, f"{out_folder}/comprehensive_report.png")

In [ ]:
# Alternative: Run complete analysis in one step
# This method does everything - detection, file outputs, and all visualizations
complete_results = detector.run_complete_analysis(
    naip_2019_path, naip_2022_path, "complete_analysis_results"
)